In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
df = pd.read_csv('data/indego/indego-trips-2019-q4.csv')

I need to reshape the data so that I have hourly observations for each station with inflow and outflow.

In [3]:
copy = df.dropna()
df = copy.copy()

In [4]:
# convert the start and stop times to 
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])


#df['start_times'] = df['start_time'].dt.floor(freq='H')
#df['end_times'] = df['end_time'].dt.ceil(freq='H')

mask = df['trip_route_category'] == 'One Way'
df.loc[mask, 'trip_route_category'] = 0

mask = df['trip_route_category'] == 'Round Trip'
df.loc[mask, 'trip_route_category'] = 1

df['trip_route_category'] = pd.to_numeric(df['trip_route_category'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163403 entries, 0 to 166115
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   trip_id              163403 non-null  int64         
 1   duration             163403 non-null  int64         
 2   start_time           163403 non-null  datetime64[ns]
 3   end_time             163403 non-null  datetime64[ns]
 4   start_station        163403 non-null  int64         
 5   start_lat            163403 non-null  float64       
 6   start_lon            163403 non-null  float64       
 7   end_station          163403 non-null  int64         
 8   end_lat              163403 non-null  float64       
 9   end_lon              163403 non-null  float64       
 10  bike_id              163403 non-null  int64         
 11  plan_duration        163403 non-null  int64         
 12  trip_route_category  163403 non-null  int64         
 13  passholder_typ

In [6]:
#mask = df['start_station'] == 3158
outdf = df.loc[:, ['trip_id', 'duration', 'start_station', 'trip_route_category', 'start_time']]

#mask = df['end_station'] == 3158
indf = df.loc[:, ['trip_id', 'duration', 'end_station', 'trip_route_category', 'end_time']]

In [7]:
indf['flow'] = 1
outdf['flow'] = -1

In [8]:
indf = indf.rename(columns={'duration' : 'incoming_duration', 'end_station' : 'station', 'end_time' : 'datetime'})

In [9]:
outdf = outdf.rename(columns={'duration' : 'outgoing_duration', 'start_station' : 'station', 'start_time' : 'datetime'})

In [10]:
stapled = pd.concat([indf, outdf])

In [11]:
stapled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326806 entries, 0 to 166115
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   trip_id              326806 non-null  int64         
 1   incoming_duration    163403 non-null  float64       
 2   station              326806 non-null  int64         
 3   trip_route_category  326806 non-null  int64         
 4   datetime             326806 non-null  datetime64[ns]
 5   flow                 326806 non-null  int64         
 6   outgoing_duration    163403 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 19.9 MB


In [ ]:
stapled['station'].nunique()

In [12]:
stapled['station'].unique()

array([3163, 3037, 3164, 3110, 3018, 3045, 3065, 3007, 3112, 3182, 3046,
       3005, 3062, 3052, 3210, 3169, 3196, 3034, 3030, 3012, 3200, 3063,
       3158, 3066, 3020, 3026, 3125, 3004, 3108, 3166, 3187, 3157, 3064,
       3099, 3041, 3119, 3056, 3022, 3054, 3088, 3015, 3161, 3195, 3038,
       3156, 3206, 3073, 3078, 3009, 3061, 3057, 3033, 3160, 3055, 3072,
       3154, 3168, 3032, 3019, 3060, 3114, 3010, 3167, 3170, 3068, 3120,
       3129, 3185, 3006, 3021, 3059, 3058, 3028, 3159, 3190, 3051, 3047,
       3152, 3014, 3031, 3197, 3040, 3029, 3202, 3102, 3050, 3100, 3101,
       3039, 3201, 3165, 3123, 3075, 3192, 3017, 3124, 3053, 3008, 3153,
       3024, 3035, 3086, 3184, 3013, 3071, 3162, 3069, 3049, 3070, 3183,
       3098, 3027, 3116, 3096, 3043, 3025, 3074, 3118, 3097, 3067, 3111,
       3115, 3121, 3113, 3155, 3117, 3181, 3011, 3016, 3209, 3188, 3107,
       3150, 3093, 3077, 3106, 3104, 3186, 3042, 3208, 3203, 3204, 3205])

In [20]:
# create a list of the stations
stations = stapled['station'].unique()

# create a dict to create and hold the dataframes
station_frames = {}

# loop through the data for each station
for station in stations:
    
    #select the data for that station
    mask = stapled['station'] == station
    small = stapled.loc[mask, :]
    
    # aggregate it
    small.set_index('datetime', inplace=True)
    grouped = small.resample('H').agg({'flow' : 'sum'
                         , 'incoming_duration' : 'mean'
                        , 'outgoing_duration' : 'mean'
                        , 'trip_route_category' : 'mean'})
    
    # preserves the int type better than .agg('mean')
    grouped['station'] = station
    
    station_frames[station] = grouped

In [25]:
unified = pd.DataFrame()
for station in stations:
    unified = pd.concat([unified, station_frames[stationa]])


In [28]:
unified[unified['station'] == 3070]

,flow,incoming_duration,outgoing_duration,trip_route_category,station
datetime,,,,,
2019-10-01 07:00:00,-1,NaN,17.000000,0.0,3070
2019-10-01 08:00:00,0,17.0,16.000000,0.0,3070
2019-10-01 09:00:00,-2,24.0,9.333333,0.0,3070
2019-10-01 10:00:00,0,NaN,NaN,NaN,3070
2019-10-01 11:00:00,1,106.0,NaN,0.0,3070
...,...,...,...,...,...
2019-12-31 20:00:00,0,NaN,NaN,NaN,3070
2019-12-31 21:00:00,0,22.0,122.000000,0.0,3070
2019-12-31 22:00:00,0,NaN,NaN,NaN,3070
